In [1]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models.ollama import ChatOllama
from langchain_objectbox.vectorstores import ObjectBox
from langchain.prompts import ChatPromptTemplate
from objectbox import Store
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import FAISS

import os
import shutil

In [21]:
EMBEDDING = OllamaEmbeddings(model="mxbai-embed-large")

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

CHAT_MODEL = ChatOllama(model='llama3.1:8b', embedding=EMBEDDING)


In [3]:
objectbox = ObjectBox(embedding=EMBEDDING, embedding_dimensions=1024)

In [33]:
def vector_search_db(query_text):
    query_embedding = EMBEDDING.embed_query(query_text)
    db_results = objectbox.similarity_search_by_vector(query_embedding, k=6)
    return db_results

def generate_prompt(db_results, query_text):
    context_text = "\n\n---\n\n".join([doc.page_content for doc in db_results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    return prompt

def generate_response(prompt):
    return CHAT_MODEL.predict(prompt)

def format_response(response_text, db_results):
    sources = set([doc.metadata.get("source", None) for doc in db_results])
    sources = [source.split("/",1)[-1].rsplit(".",1)[0] for source in sources if source is not None]
    return f"Response: {response_text}\nSources: {sources}"

In [36]:
retriever = objectbox.as_retriever()
query = "What are the duties of the secretary of state?"
db_results = retriever.invoke(query, k=6)
print(len(db_results), "results found.")
prompt = generate_prompt(db_results, query)
response = generate_response(prompt)
formatted_response = format_response(response, db_results)
print(formatted_response)

4 results found.
Response: According to the provided context, the duties of the Secretary of State include:

*   Administration and enforcement of immigration and nationality laws
*   Determination of nationality of a person
*   Functions concerning United States passports (with respect to citizens of the United States)
*   Amendment or revocation of part 46 of title 22, Code of Federal Regulations (concerning persons other than citizens of the United States)
*   Administration and enforcement of provisions of this chapter and all other immigration and nationality laws relating to diplomatic and consular officers of the United States.
Sources: ['8 USC 1185 Travel control of citizens and aliens', '8 USC 1104 Powers and duties of Secretary of State']


In [34]:
query = "What are the duties of the secretary of state?"
db_results = vector_search_db(query)
# for doc in db_results:
#     print(doc.metadata.get("source", None).split("/", 1)[-1])
if db_results:
    prompt = generate_prompt(db_results, query)
    response = generate_response(prompt)
    print(format_response(response, db_results))

Response: According to Section 1104 of the United States Code, the duties of the Secretary of State include:

* Administering and enforcing the provisions of Chapter 1104 and all other immigration and nationality laws relating to:
	+ The powers, duties, and functions of diplomatic and consular officers of the United States (except for those related to granting or refusing visas)
	+ The powers, duties, and functions of the Administrator
	+ The determination of nationality of a person
* Exercising any authority vested by law in any official or office of the Department of State
* Maintaining close liaison with the appropriate committees of Congress regarding the administration of this chapter by consular officers
* Designating an Administrator who shall be responsible for:
	+ Maintaining close liaison with the appropriate committees of Congress regarding the administration of this chapter by consular officers
	+ Performing duties delegated to them by the Secretary of State
Sources: ['8 US